In [2]:
import pandas as pd

In [60]:
# Load our file
try:
    with open("noahs-customers.csv", "r") as rfp:
        df = pd.read_csv(rfp)
except FileNotFoundError:
    print("Failed to open the file.")


# Day 1

In [123]:
t9_lookup = {
    "ABC" : "2",
    "DEF" : "3",
    "GHI" : "4",
    "JKL" : "5",
    "MNO" : "6",
    "PQRS" : "7",
    "TUV" : "8",
    "WXYZ" : "9",
}

def is_char_in_key(char: str, key: str) -> bool:
    return char.upper() in key

def format_phone_number(phone_number_string: str) -> str:
    return f"{phone_number_string[:3]}-{phone_number_string[3:6]}-{phone_number_string[6:]}"



def row_to_phone(row) -> str:
    last_name = "".join(row['name'].split(" ")[1:])
    if len(last_name) != 10:
        return ""
    last_name = last_name.replace(" ", "")
    output_phone_num_str = ""
    for character in last_name:
        for key in t9_lookup.keys():
            if is_char_in_key(character, key):
                output_phone_num_str += t9_lookup[key]
                break
    return format_phone_number(output_phone_num_str)



## Name -> Phone lookup
First, we want to try and match the person's name and their phone number using the convention:
```
ABC     : 2
DEF     : 3
GHI     : 4
JKL     : 5
MNO     : 6
PQRS    : 7
TUV     : 8
WXYZ    : 9
<space> : 0
```

In [ ]:
# Show just the interesting columns

sub_df = df[['name', 'phone']]
sub_df['name_to_phone'] = sub_df.apply(row_to_phone, axis=1)
sanitized = sub_df.query("name_to_phone != '' ")
sanitized['res'] = sanitized.phone == sanitized.name_to_phone


result = sanitized.loc[sanitized['res'] == True]
print(result)


# Day 2

We're looking for soneone with initials JD in 2017, who bought coffee and bagels.

In [8]:
# Load three dataframes that we intend to cross over
try:
    with open("noahs-orders_items.csv", "r") as rfp:
        orders_items_df = pd.read_csv(rfp)
except FileNotFoundError:
    print("Failed to open the file.")

try:
    with open("noahs-orders.csv", "r") as rfp:
        orders_df = pd.read_csv(rfp)
except FileNotFoundError:
    print("Failed to open the file.")

try:
    with open("noahs-products.csv", "r") as rfp:
        products_df = pd.read_csv(rfp)
except FileNotFoundError:
    print("Failed to open the file.")

## Find SKUs for Bagels and Coffee items

In [45]:
[products_df.query(f"desc == '{x}'") for idx, x in enumerate(products_df['desc']) if "coffee" in x.lower()]

# Out of all these products we can see that Coffe, Drip is with SKU DLI1464.
# Now for the bagels

[products_df.query(f"desc == '{x}'") for idx, x in enumerate(products_df['desc']) if "bagel" in x.lower()]

# For bagels we have two interesting SKUs, BKY4234 and BKY5887.
# Therefore,

important_skus = ["DLI1464", "BKY4234", "BKY5887"]


## Find orderID for transactions involving these SKUS, in 2017

In [ ]:
important_orders = orders_items_df[orders_items_df['sku'].isin(important_skus)]['orderid'].tolist()
print(len(important_orders))
important_orders

## Find customers by orderID

In [95]:
important_customers = orders_df[orders_df['orderid'].isin(important_orders)]['customerid'].tolist()


## Find customer names


In [ ]:
customers = df[df["customerid"].isin(important_customers)]['name']
customer_list = customers.tolist()

def find_customer_with_initials(customer_list: list, initials: str) -> list:
    names = []
    for customer in customer_list:
        split_name = customer.split(" ")
        if len(split_name) != 2:
            continue
        customer_initials = split_name[0][0] + split_name[1][0]
        if customer_initials == initials.upper():
            names.append(customer)
    return names

customer_names = find_customer_with_initials(customer_list, "JD")

phones = df[df['name'].isin(customer_names)]['phone']
print(phones)